In [15]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from clean_data import clean_data, prepare_data
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [16]:
# Read files
train = pd.read_csv('../cars-competition/data/cars_train.csv', index_col='Id')
submission = pd.read_csv('../cars-competition/data/cars_test.csv', index_col='Id')

In [17]:
# Clean Data
train = clean_data(train)
submission = clean_data(submission)

Cleaning cylinders...
Cleaning condition...
Cleaning odometer...
Cleaning drive...
Cleaning size...
Cleaning manufacturer...
Cleaning fuel...
Cleaning transmission...
Cleaning title status...
Data cleaning complete!
Cleaning cylinders...
Cleaning condition...
Cleaning odometer...
Cleaning drive...
Cleaning size...
Cleaning manufacturer...
Cleaning fuel...
Cleaning transmission...
Cleaning title status...
Data cleaning complete!


In [113]:
# Prepare Data
# columns = ['year','manufacturer','condition','cylinders','fuel','odometer','title_status','transmission','drive','size']
columns = ['year','manufacturer','condition','cylinders','odometer','title_status','transmission','size']
X, y = prepare_data(train,columns,typ='train')
X_sub, _ = prepare_data(submission,columns,typ='test')

In [114]:
# Feature reduction PCA
n_components = [5,10,15,20,25,30,35,40]
for n in n_components:
    exec('pca{0} = PCA(n_components = {0})'.format(n))
    exec('X_new{0} = pca{0}.fit_transform(X)'.format(n))

In [115]:
# Split Train and Test

for n in n_components:
    exec('X_train{0}, X_test{0}, y_train{0}, y_test{0} = train_test_split(X_new{0},y,test_size=0.2)'.format(n))

In [116]:
# Linear Regression
for n in n_components:
    exec('lin_reg{0} = LinearRegression()'.format(n))
    exec('lin_reg{0}.fit(X_train{0},y_train{0})'.format(n))

In [117]:
# Predict
for n in n_components:
    exec('y_pred{0} = lin_reg{0}.predict(X_test{0})'.format(n))

In [118]:
# Check error
error = {}
for n in n_components:
    exec('error[{0}] = mean_squared_error(y_test{0}, y_pred{0})'.format(n))

In [119]:
# Check for optimal feature number for PCA: 25
error

{5: 59482671630874.11,
 10: 92412197469180.89,
 15: 130381821570407.27,
 20: 107064290225244.83,
 25: 68743885738660.945,
 30: 35383609850520.66,
 35: 22276561194830.145,
 40: 85254343998279.33}

In [120]:
# Try optimal Linear Regression for submission
# PCA
X_sub35 = pca35.transform(X_sub)
# Predict
y_sub35 = lin_reg35.predict(X_sub35)
y_sub35

array([ 15207.9483823 , 394082.58502302,  39146.55895167, ...,
       558152.55218327, 233902.25240339, 299847.45644746])

In [121]:
# Getting absolute values
y_sub35 = abs(y_sub35)
y_sub35

array([ 15207.9483823 , 394082.58502302,  39146.55895167, ...,
       558152.55218327, 233902.25240339, 299847.45644746])

In [122]:
# Prepare for submission
sub35 = submission
sub35['price'] = y_sub35
sub35 = sub35['price'] 
sub35.to_csv('../cars-competition/data/sub35_ln_bsf.csv', header=True, index=True)